In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools

In [2]:
folder = ""
raw = folder + "raw_data/"
raw_dataset_x = pd.read_csv(raw + 'X_train.csv',index_col=['id'])


In [3]:
raw_dataset_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5117 entries, 0 to 5116
Columns: 17807 entries, x0 to x17806
dtypes: float64(15348), int64(2459)
memory usage: 695.2 MB


In [4]:
raw_dataset_y = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
raw_dataset_y.head()

,y
id,
0,0
1,0
2,0
3,1
4,2


In [5]:
just_scoring = True
if just_scoring:
    preprocessed = folder + "preprocessed_data/"
    #x_train, x_test, y_train, y_test = train_test_split(raw_dataset_x, raw_dataset_y, test_size=0.2, random_state=100)
    x_train = pd.read_csv(raw + 'X_train.csv', index_col=['id'])
    x_test = pd.read_csv(raw + 'X_test.csv', index_col=['id'])
    y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
    y_test = pd.DataFrame({'A': []})
else:
    preprocessed = folder + "ready_data/"
    x_train = pd.read_csv(raw + 'X_train.csv', index_col=['id'])
    x_test = pd.read_csv(raw + 'X_test.csv', index_col=['id'])
    y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
    y_test = pd.DataFrame({'A': []})

In [9]:
def apply_func(index,row):
    a = datetime.datetime.now()
    ecg_signal = row.dropna().to_numpy()
    ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=300)
    #     ecg_cleaned = ecg_signal

    # to try
    #max_abs = max(ecg_cleaned.min(), ecg_cleaned.max(), key=abs)
    #ecg_cleaned = ecg_cleaned / max_abs
    #     # Extract R-peaks locations
    peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)
    '''
    #     RR_intervals = np.diff(rpeaks['ECG_R_Peaks'])
    #     row = add_basic_info(row, RR_intervals, 'RR_intervals')

    #     R_amplitudes = np.array([ecg_cleaned[i] for i in rpeaks['ECG_R_Peaks']])
    #     row = add_basic_info(row, R_amplitudes, 'R_amplitudes')

    #     if len(rpeaks['ECG_R_Peaks']) > 3:
    #         hrv_time = nk.hrv_time(peaks, sampling_rate=300, show=False)
    #         for index, hrv_time_row in hrv_time.iterrows():
    #             for col in hrv_time.columns:
    #                 row[col] = hrv_time_row[col]

    #         hrv_freq = nk.hrv_frequency(peaks, sampling_rate=300, show=False, normalize=True)
    #         for index, hrv_freq_row in hrv_freq.iterrows():
    #             for col in hrv_freq.columns:
    #                 row[col] = hrv_freq_row[col]

    #         hrv_non = None
    #         try:
    #             hrv_non = nk.hrv_nonlinear(peaks, sampling_rate=300, show=False)
    #         except ValueError:
    #             print("Got ValueError")
    #         if hrv_non is not None:
    #             for index, hrv_non_row in hrv_non.iterrows():
    #                 for col in hrv_non.columns:
    #                     row[col] = hrv_non_row[col]

    #     out = ecg.ecg(signal=ecg_signal, sampling_rate=300., show=False)
    #     heartbeats = out['templates']
    #     max_abs = max(heartbeats.min(), heartbeats.max(), key=abs)
    #     heartbeats = heartbeats / max_abs
    #     mu = np.mean(heartbeats, axis=0)
    #     std = np.std(heartbeats, axis=0)
    #     row["max_of_std_Heartbeats"] = np.max(std)
    #     row["mean_of_std_Heartbeats"] = np.mean(std)
    #     row["min_of_std_Heartbeats"] = np.min(std)
    #     row["range_of_std_Heartbeats"] = np.max(std) - np.min(std)
    '''
    # Delineate the ECG signal
    b = datetime.datetime.now()
    #print("r_peak", (b - a).seconds)
    values_dict = {}
    nk_features = np.zeros((0,14))
    if len(rpeaks['ECG_R_Peaks']) > 3:
        try:
            signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned, rpeaks, show_type='all', sampling_rate=300,
                                                     method="dwt")
            for key, arr in waves_dwt.items():
                if 'Onsets' in key or 'Offsets' in key:
                    array = np.array(arr)
                    #row = add_basic_info(row, np.array(arr), key)
                    #row = add_basic_info(row, amplitudes, f"{key}_amplitudes")
            nk_features = get_nk_features(ecg_cleaned, rpeaks['ECG_R_Peaks'], waves_dwt)
        except:
            ecg_signal = row.dropna().to_numpy()
            ecg_signal = -ecg_signal
            ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=300)
            peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)
            signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned, rpeaks, show_type='all', sampling_rate=300,
                                                     method="dwt")
            for key, arr in waves_dwt.items():
                if 'Onsets' in key or 'Offsets' in key:
                    array = np.array(arr)
                    #row = add_basic_info(row, np.array(arr), key)
            nk_features = get_nk_features(ecg_cleaned, rpeaks['ECG_R_Peaks'], waves_dwt)
            
    mu = np.zeros(180)
    median = np.zeros(180)
    std = np.zeros(180)
    if len(rpeaks['ECG_R_Peaks']) >= 2:
        # print(ecg.extract_heartbeats(signal, r_peaks, 300))

        beats = ecg.extract_heartbeats(ecg_signal, rpeaks['ECG_R_Peaks'], 300)['templates']

        if len(beats) != 0:
            mu = np.mean(beats, axis=0)
            median = np.median(beats, axis=0)
            std = np.std(beats, axis=0)

    c = datetime.datetime.now()
    #print("extract_heartbeats", (c - b).seconds)

    d = datetime.datetime.now()
    #print("delineate", (d - c).seconds)
    fft_window = np.fft.fft(ecg_cleaned)
    freq = np.fft.fftfreq(np.array(ecg_cleaned).shape[-1], d=1/300)
    fft_ps = np.abs(fft_window)**2
    
    #print(fft_window, freq, fft_ps)
    e = datetime.datetime.now()
    #print("fft", (e - d).seconds)
    row = row.drop(columns=[f'x{num}' for num in range(17807)])
    f = datetime.datetime.now()
    
    #print("drop", (f - e).seconds)
    for i in range(180):
        values_dict[f'mu_{i}'] = mu[i]
        values_dict[f'median_{i}'] = median[i]
        values_dict[f'std_{i}'] = std[i]
    for i in range(len(fft_window)):
        #values_dict[f'fft_window_{i}'] = fft_window[i]
        #values_dict[f'freq_{i}'] = freq[i]
        #values_dict[f'fft_ps_{i}'] = fft_ps[i]
        pass
    for i in range(len(nk_features)):
        values_dict[f'nk_feat_{i}'] = nk_features[i]
    
    #row.append(pd.Series(values_dict))
    g = datetime.datetime.now()
    #print("add", (g - f).seconds)
    return values_dict

In [10]:
#classify ecg
from functions import *
def classify_ecg(x_train):
    x_train_ecg = x_train.copy()
    '''
    x_train_ecg = x_train_ecg.apply(apply_func)
    '''
    new_train_dict = {}
    for index,row in x_train.iterrows():
        new_train_dict[str(index)] = apply_func(index,row)
        if len(new_train_dict) % 1000 == 0:
            print(len(new_train_dict))
        
    return pd.DataFrame.from_dict(new_train_dict,orient='index')
    
    return x_train_ecg

In [11]:
x_train = classify_ecg(x_train)

/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

1000


/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

2000


/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

...

/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

4000


/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

5000


/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

In [12]:
x_train.to_csv('x_train_preprocess.csv')

In [13]:
x_test = classify_ecg(x_test)

/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

1000


/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

2000


/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

3000


/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/

In [ ]:
x_test.to_csv('x_test_preprocess.csv')

In [ ]:
#count nan values per column
def count_nan(df):
    for col in df.columns:
        if(df[col].isna().sum() > 0):
            print(col,df[col].isna().sum())
    
count_nan(x_train)

In [ ]:
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [ ]:
x_train.info()

In [ ]:
selector = SelectKBest(score_func=f_classif, k=150)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

In [ ]:
#print column names of selected features
x_train.columns[cols].tolist()

In [ ]:
#x_train= x_train.iloc[:,cols]
#x_test = x_test.iloc[:,cols]

In [ ]:
#scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
class_weights = {}
for c in range(4):
    class_weights.update({c:len(y_train) / float(np.count_nonzero(y_train == c))})


svm_model = svm.SVC( kernel='rbf', degree=3, gamma='auto',class_weight='balanced', decision_function_shape='ovo', random_state=0)

#grid search

param_grid = {'C': [0.1, 1, 10, 100, 1000],
                'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001]}
#grid = GridSearchCV(svm_model, param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_macro', cv=5)
#grid.fit(x_train, np.ravel(y_train))
svm_model.fit(x_train, np.ravel(y_train))

In [ ]:
grid.best_params_

In [ ]:
y_pred_svm = grid.predict(x_test)

In [ ]:
#Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
mod = GradientBoostingClassifier(n_estimators=100, random_state=0).fit(x_train, np.ravel(y_train))

In [ ]:
y_pred_gbc = mod.predict(np.array(x_test))

In [ ]:
from sklearn.metrics import f1_score
def performance(y_pred):
    varb = y_test.empty
    if not y_test.empty:
        return f1_score(y_test, y_pred, average='macro')

In [ ]:
performance(y_pred_gbc)
#0.5124307581481997 120 features without fft and std
#0.5314063935937154 all features without fft and std
#0.5424081408715644 all features with fft
#

In [ ]:
performance(y_pred_svm)
#0.5127457326409542 120 features without fft and std
#0.5465124489502252 all features without fft and std
#0.533805355742844 all features with fft
#0.5166443399222037 all features with fft with grid

In [ ]:
#multi-class classification with extra trees classifier
from sklearn.ensemble import ExtraTreesClassifier
extra_t = ExtraTreesClassifier(n_estimators=500, random_state=0)
extra_t.fit(x_train, np.ravel(y_train))
y_pred_extra = extra_t.predict(x_test)


In [ ]:
performance(y_pred_extra)
#0.4157898314356613 120 features without fft and std
#0.414489522417154 all features without fft and std
#0.5278405092444501 all features with fft

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="multi:softmax", random_state=42, eval_metric=f1_score)
xgb_model.fit(x_train, np.ravel(y_train))
y_pred_xgb = xgb_model.predict(x_test)

In [ ]:
performance(y_pred_xgb)
#0.5203003460330339 all features without fft and std
#0.59181224550077 all features with fft

In [ ]:
#classification with random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, random_state=0,class_weight = 'balanced')
rf.fit(x_train, np.ravel(y_train))
y_pred_rf = rf.predict(x_test)

In [ ]:
performance(y_pred_rf)
#0.4124245169082126 120 features without fft and std
#0.4347590188338277 all features without fft and std
#0.4752277078666676 all features with fft

In [ ]:
#classification with catboost
from catboost import CatBoostClassifier
cat = CatBoostClassifier(objective= 'MultiClass',iterations=500, random_state=0, thread_count = 10,eval_metric='TotalF1')

#grid search catboost
grid_catbooost = {'iterations': [300],
        'l2_leaf_reg': [0.05, 0.1, 0.2, 0.5, 1, 2]}
cat.fit(x_train, np.ravel(y_train))

In [ ]:
y_pred_cat = cat.predict(x_test)

In [ ]:
performance(y_pred_cat)
#0.5554033483943467 120 features without fft and std
#0.5480459556546513 all features without fft and std
#0.6047056861228775 all features with fft
#0.5166443399222037 all features with fft with grid

In [ ]:
#stacking classifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
estimators = [('svm', svm_model), ('gbc', mod), ('extra', extra_t), ('xgb', xgb_model), ('rf', rf), ('cat', cat)]
final_estimator = LogisticRegression()
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator, cv=5, n_jobs = -1)

In [ ]:
clf.fit(x_train, np.ravel(y_train))
y_pred_stack = clf.predict(x_test)

In [ ]:
performance(y_pred_stack)

In [ ]:
pd.DataFrame(y_pred_stack,columns = ['y']).to_csv('output_stacking.txt',index_label="id")